In [62]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import random
import os
import cv2 as cv

In [63]:
filenames = os.listdir("train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [92]:
for i in range(100):
    df = df.sample(frac=1).reset_index(drop=True)
df

,filename,category
0,dog.2027.jpg,1
1,dog.827.jpg,1
2,dog.5255.jpg,1
3,cat.5050.jpg,0
4,dog.10725.jpg,1
...,...,...
24995,dog.6791.jpg,1
24996,dog.10389.jpg,1
24997,cat.5327.jpg,0
24998,dog.8718.jpg,1


In [114]:
data_train = []
for i in range(25000):
    label = df['category'][i]
    img = cv.imread("train/"+df['filename'][i],cv.COLOR_BGR2RGB)
    data_train.append([np.array(img),np.array(label)])
#np.save('train_data.npy', data_train)

In [115]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(1, activation='sigmoid')) # 2 because we have cat and dog classes

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [116]:
X = np.array([i[0] for i in data_train]).reshape(-1,128,128,3)
Y = np.array([i[1] for i in data_train])

In [117]:
X = X.astype('float32')
X = X / 255.0

In [118]:
model.fit(X, Y, epochs=3, batch_size = 128)

Epoch 1/3
196/196 [==============================] - 712s 4s/step - loss: 0.7174 - accuracy: 0.6333
Epoch 2/3
196/196 [==============================] - 701s 4s/step - loss: 0.5587 - accuracy: 0.7152
Epoch 3/3
196/196 [==============================] - 724s 4s/step - loss: 0.4837 - accuracy: 0.7634


In [119]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_40 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_41 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 30, 30, 64)      

In [120]:
model.save('dogVScat.model')

INFO:tensorflow:Assets written to: dogVScat.model\assets


In [121]:
val_loss, val_acc = model.evaluate(X, Y)
print(val_loss)
print(val_acc)

782/782 [==============================] - 135s 172ms/step - loss: 0.5203 - accuracy: 0.7339
0.5203279852867126
0.7339199781417847


In [153]:
np.argmax(model.predict(X[13].reshape(-1,128,128,3)))

0

In [151]:
Y[13]

1